In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.utils import model_to_dot, plot_model
from IPython.display import clear_output
from keras.layers import Reshape
from keras.layers import GlobalMaxPooling1D
from PIL import Image
import os
import numpy as np
from bayes_opt import BayesianOptimization
import time
import pandas as pd

#Create the input node. We omit batch size when using functional API
inputs = keras.Input(shape = (672, 1))


#Then we implement the convolution layer
con_1D = keras.layers.Conv1D(filters=64, kernel_size=3)(inputs)
print(f'convolution layer shape:', con_1D.shape)

#Then we construct the parallel bilstm layer
biLSTM = Bidirectional(LSTM(100,return_sequences=True))(inputs)
biLSTM= Dropout(0.2)(biLSTM)
print(f'bilstm layer shape:', biLSTM.shape)

# ADDING PADDING SEQUENCE TO MAKE THEM COMPATIBLE BEFORE CONCATENATION

#first we compute the length difference between the two outputs
pad_difference = (biLSTM.shape[1] - con_1D.shape[1])

#then we implement a for loop to ensure padding to equal lenth even if odd lenth difference
#// double division is to give us an integer rather than a float output
if (pad_difference) % 2 == 0:
    pad_1, pad_2 = pad_difference // 2, pad_difference // 2
else:
    pad_1, pad_2 = pad_difference // 2, (pad_difference // 2) + 1

#Now applying the padding to the convolution layer
padding_layer = keras.layers.ZeroPadding1D(padding=(pad_1, pad_2))(con_1D)
print(f'padding layer shape:', padding_layer.shape)

#Then we merge
merged = keras.layers.Concatenate()([padding_layer,biLSTM])
print(f'merged layer shape:', merged.shape[1])

#Then we reshape such that steps is the last item on that list
reshaped = Reshape((264, 672)) (merged)
print(f'reshaped layer shape:', reshaped.shape)
#Then we add a bilstm to condense the 264 variants into 1. Note, unlike in first bilstm, return-sequence set to false by not being declared
max_pool_1D = GlobalMaxPooling1D()(reshaped)
print(f'max_pool_1D layer shape:', max_pool_1D.shape)

#Then the final output layer
output_layer = Dense(48)(max_pool_1D)
print(f'output_layer layer shape:', output_layer.shape)

# #Then pull everything together to build the final model
# model = keras.models.Model(inputs=inputs, outputs=output_layer)


convolution layer shape: (None, 670, 64)
bilstm layer shape: (None, 672, 200)
padding layer shape: (None, 672, 64)
merged layer shape: 672
reshaped layer shape: (None, 264, 672)
max_pool_1D layer shape: (None, 672)
output_layer layer shape: (None, 48)
